<a href="https://colab.research.google.com/github/JekugaBrandon/classification_with_tensor_all/blob/master/classification_with_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.18.0


In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [17]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


In [18]:
# batching the datasets

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ads = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [19]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [23]:
#Testing the models

@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)


  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [26]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)


  for test_images, test_labels in test_ads:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result():0.2f}, '
      f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
      f'Test Loss: {test_loss.result():.2f}, '
      f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )

Epoch 1, Loss: 0.13, Accuracy: 96.00, Test Loss: 0.07, Test Accuracy: 97.88
Epoch 2, Loss: 0.04, Accuracy: 98.61, Test Loss: 0.06, Test Accuracy: 98.26
Epoch 3, Loss: 0.02, Accuracy: 99.30, Test Loss: 0.07, Test Accuracy: 97.89
Epoch 4, Loss: 0.01, Accuracy: 99.51, Test Loss: 0.06, Test Accuracy: 98.33
Epoch 5, Loss: 0.01, Accuracy: 99.72, Test Loss: 0.07, Test Accuracy: 98.47
